In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.0 MB/s eta 0:00:00


In [ ]:
import glob
import os
import requests
import time
import warnings

warnings.simplefilter('ignore', FutureWarning)

from collections import defaultdict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm.auto import tqdm

import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary

import torchvision
from torchvision import transforms

from IPython.display import clear_output
%matplotlib inline

sns.set(font_scale=1.2)
# sns.set_style(style='whitegrid')
device_num = 0
#torch.cuda.set_device(device_num)#####################################################
from google.colab import drive
import shutil
import random


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
tqdm.pandas()

In [ ]:
drive.mount('/content/drive')

In [ ]:
shutil.copy('/content/drive/MyDrive/CURS_22_DF/19_12_scale_df_signals.pkl', '/content/',  )

In [ ]:
df = pd.read_pickle('19_12_scale_df_signals.pkl')
df

In [ ]:
def noise( x, N):
        snr_dB=N
        snr = 10**(snr_dB/10)
        sigmaN2 = 1/snr

        # generate AWGN
        n = (sigmaN2**0.5)*(np.random.randn(len(x)) + 1j*np.random.randn(len(x)))
        return x + n

def make_noise(df, N):
    df_w = df.copy()
    df_w['signal_IQ'] = df_w['signal_IQ'].apply(lambda x: noise(x, N))
    return df_w

def moment(x,p,q):
    return np.mean(x**(p-q) * x.conjugate()**q)

def cumulant(x,p,q):
        if p == 2:
            return moment(x,p,q)
        elif p == 4:
            if q == 0:
                return moment(x,4,0) - 3 * moment(x,2,0) **2
            elif q == 1:
                return moment(x,4,1) - 3 * moment(x,2,1) **2
            return moment(x,4,2) - moment(x,2,0).real**2 - 2 * moment(x,2,1) **2

def make_mom_cum(df):
    df_out = df.copy()
    df_cols_name = []
    moments_num = [[1, 0], [2, 0], [2, 1], [3, 0], [3, 1], [3, 2], [4, 0]]
    for num in (moments_num):
        df_cols_name.append(f'moment_{num[0]}_{num[1]}')
        df_out[df_cols_name[-1]] = df_out.signal_IQ.apply(lambda x: moment(x,num[0], num[1]))
    cumulants_num = [(2,0),(2,1),(4,0),(4,1),(4,2)]
    for num in (cumulants_num):
        df_cols_name.append(f'cumulants_{num[0]}_{num[1]}')
        df_out[df_cols_name[-1]] = df_out.signal_IQ.apply(lambda x: cumulant(x,num[0], num[1]))

    return df_out

In [ ]:
#list_SNR = sorted(list( set(list(range(-30,31,5)) + list(range(-10,11,1))) ))
list_SNR = sorted(list(range(-30,31,1)))
print(len(list_SNR))
#list_SNR

In [ ]:
models = {'CatBoostClassifier':CatBoostClassifier(silent=True, iterations = 800 ),
          'RandomForestClassifier':RandomForestClassifier(),
          'ExtraTreesClassifier':ExtraTreesClassifier(),
          'DecisionTreeClassifier':DecisionTreeClassifier(),
          'MultinomialNB':MultinomialNB(),
          'SVC':SVC(),
          'LogisticRegression':LogisticRegression(),
          'KNeighborsClassifier':KNeighborsClassifier()
}

In [ ]:
class digitsPerceptron(nn.Module):
    acrivete_func = nn.Softsign()
    def __init__(self, input_dim=1024, hidden_dim=100):
        super().__init__()
        self.n_classes = 8
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            self.acrivete_func,
            nn.Linear(hidden_dim, hidden_dim),
            self.acrivete_func,
            # nn.Linear(hidden_dim, hidden_dim),
            # self.acrivete_func,
            nn.Linear(hidden_dim, self.n_classes),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        return self.net(x)
device = f"cuda:{device_num}" if torch.cuda.is_available() else "cpu"
#device = "cpu"
device

'cuda:0'

In [ ]:
%%time
df_acc = pd.DataFrame()
for SNR in tqdm(range(-30,31)):
    noise_sig = make_noise(df,SNR)
    df_train = make_mom_cum(noise_sig)
    X = df_train.drop(['type_sig','signal_IQ'],axis =1)
    y = df_train['type_sig']
    le = preprocessing.LabelEncoder()
    X = X.applymap(lambda x: x.real)
    
    y1 = le.fit_transform(y)
    X_train_in, X_test_in, y_train_in, y_test_in = train_test_split(
        X, y1, test_size=0.2, random_state=21,
        shuffle=True, stratify=y
    )
    scaler = MinMaxScaler()
    X_train_in = scaler.fit_transform(X_train_in)
    X_test_in = scaler.transform(X_test_in)
    df_add = pd.DataFrame()
    for key, model in tqdm( models.items() ):

        
        #df_add.loc[key,'model'] = key
        model.fit(X_train_in,y_train_in)
        y_pred = model.predict(X_test_in)
        df_add.loc[key,SNR] = accuracy_score(y_test_in, y_pred)
    #####################################    
    model = digitsPerceptron(            
            input_dim=len(X.columns), 
            hidden_dim=100
        ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    # X_train_in.reset_index(drop=True,inplace = True)
    # X_test_in.reset_index(drop=True,inplace = True)     
    X_train = torch.tensor(X_train_in, dtype=torch.float).to(device)
    y_train = torch.tensor(y_train_in, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test_in, dtype=torch.float).to(device)
    y_test = torch.tensor(y_test_in, dtype=torch.long).to(device)

    EPOCHS  = 300
    loss_list = np.zeros((EPOCHS,))
    accuracy_list = np.zeros((EPOCHS,))
    for epoch in tqdm(range(EPOCHS)):
        model.train()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        loss_list[epoch] = loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test)
            correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.float)
            accuracy_list[epoch] = correct.mean().item() 
    model.eval()

    with torch.no_grad():
        y_pred = np.argmax(model(X_test).detach().cpu().numpy(), axis=1)
    #####################################
    #df_add.loc['neiro','model'] = 'neiro'
    
    df_add.loc['neiro',SNR] = accuracy_score(y_test_in, y_pred)

    df_acc = pd.concat([df_acc, df_add],axis = 1 )
    df_acc.to_csv('all_model_acc_05_01.csv')
        
    


In [ ]:
df_acc

,model,-30,model,-29,model,-28,model,-27,model,-26,...,model,26,model,27,model,28,model,29,model,30
CatBoostClassifier,CatBoostClassifier,0.128750,CatBoostClassifier,0.120417,CatBoostClassifier,0.124167,CatBoostClassifier,0.127708,CatBoostClassifier,0.125417,...,CatBoostClassifier,0.989792,CatBoostClassifier,0.990833,CatBoostClassifier,0.991875,CatBoostClassifier,0.993333,CatBoostClassifier,0.991458
RandomForestClassifier,RandomForestClassifier,0.130625,RandomForestClassifier,0.132292,RandomForestClassifier,0.123958,RandomForestClassifier,0.124167,RandomForestClassifier,0.123958,...,RandomForestClassifier,0.988333,RandomForestClassifier,0.988958,RandomForestClassifier,0.990208,RandomForestClassifier,0.988542,RandomForestClassifier,0.989167
ExtraTreesClassifier,ExtraTreesClassifier,0.124167,ExtraTreesClassifier,0.125833,ExtraTreesClassifier,0.121250,ExtraTreesClassifier,0.132083,ExtraTreesClassifier,0.132708,...,ExtraTreesClassifier,0.990208,ExtraTreesClassifier,0.989583,ExtraTreesClassifier,0.990833,ExtraTreesClassifier,0.990208,ExtraTreesClassifier,0.990000
DecisionTreeClassifier,DecisionTreeClassifier,0.126875,DecisionTreeClassifier,0.124375,DecisionTreeClassifier,0.129167,DecisionTreeClassifier,0.122917,DecisionTreeClassifier,0.125833,...,DecisionTreeClassifier,0.978958,DecisionTreeClassifier,0.980208,DecisionTreeClassifier,0.979583,DecisionTreeClassifier,0.979583,DecisionTreeClassifier,0.978333
MultinomialNB,MultinomialNB,0.126875,MultinomialNB,0.125625,MultinomialNB,0.120833,MultinomialNB,0.123542,MultinomialNB,0.123750,...,MultinomialNB,0.579167,MultinomialNB,0.576667,MultinomialNB,0.578958,MultinomialNB,0.580417,MultinomialNB,0.577708
SVC,SVC,0.125833,SVC,0.126250,SVC,0.128125,SVC,0.116250,SVC,0.123125,...,SVC,0.986458,SVC,0.985625,SVC,0.985833,SVC,0.985625,SVC,0.986042
LogisticRegression,LogisticRegression,0.126458,LogisticRegression,0.124375,LogisticRegression,0.123333,LogisticRegression,0.121875,LogisticRegression,0.124583,...,LogisticRegression,0.759167,LogisticRegression,0.809167,LogisticRegression,0.779792,LogisticRegression,0.803958,LogisticRegression,0.808125
KNeighborsClassifier,KNeighborsClassifier,0.128542,KNeighborsClassifier,0.133333,KNeighborsClassifier,0.128333,KNeighborsClassifier,0.128333,KNeighborsClassifier,0.128750,...,KNeighborsClassifier,0.990833,KNeighborsClassifier,0.990000,KNeighborsClassifier,0.989375,KNeighborsClassifier,0.990625,KNeighborsClassifier,0.990208
neiro,neiro,0.125833,neiro,0.126250,neiro,0.125000,neiro,0.120417,neiro,0.130833,...,neiro,0.756042,neiro,0.746042,neiro,0.774167,neiro,0.795208,neiro,0.729375


In [ ]:
df_acc.to_csv('all_model_acc_05_01.csv')

In [ ]:
from google.colab import files
files.download('all_model_acc_05_01.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_all1 = pd.DataFrame()
for col in df_acc.columns[1:]:
    df_all1 = pd.concat([df_all1,df_acc[col].dropna()],axis = 1)
df_all1

,-30,model,model,model,model,model,model,model,model,model,...,model,model,model,model,model,model,model,model,model,30
CatBoostClassifier,0.128750,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,...,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,CatBoostClassifier,0.991458
RandomForestClassifier,0.130625,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,...,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,0.989167
ExtraTreesClassifier,0.124167,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,...,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,ExtraTreesClassifier,0.990000
DecisionTreeClassifier,0.126875,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,...,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,DecisionTreeClassifier,0.978333
MultinomialNB,0.126875,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,...,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,MultinomialNB,0.577708
SVC,0.125833,SVC,SVC,SVC,SVC,SVC,SVC,SVC,SVC,SVC,...,SVC,SVC,SVC,SVC,SVC,SVC,SVC,SVC,SVC,0.986042
LogisticRegression,0.126458,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,...,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,LogisticRegression,0.808125
KNeighborsClassifier,0.128542,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,...,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,KNeighborsClassifier,0.990208
neiro,0.125833,neiro,neiro,neiro,neiro,neiro,neiro,neiro,neiro,neiro,...,neiro,neiro,neiro,neiro,neiro,neiro,neiro,neiro,neiro,0.729375


In [ ]:
df_all1.to_csv('all_model_acc.csv')

In [ ]:
class digitsPerceptron(nn.Module):
    acrivete_func = nn.Softsign()
    def __init__(self, input_dim=512, hidden_dim=100):
        super().__init__()
        self.n_classes = 8
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            self.acrivete_func,
            nn.Linear(hidden_dim, hidden_dim),
            self.acrivete_func,
            nn.Linear(hidden_dim, hidden_dim),
            self.acrivete_func,
            nn.Linear(hidden_dim, hidden_dim),
            self.acrivete_func,
            # nn.Linear(hidden_dim, hidden_dim),
            # self.acrivete_func,
            nn.Linear(hidden_dim, self.n_classes),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        return self.net(x)
device = f"cuda:{device_num}" if torch.cuda.is_available() else "cpu"
#device = "cpu"
device

'cuda:0'

In [ ]:
%%time
neiro_acc = {}
for SNR in tqdm(range(-30,31)):
    noise_sig = make_noise(df,SNR)
    df_train = make_mom_cum(noise_sig)
    X = df_train.drop(['type_sig','signal_IQ'],axis =1)
    y = df_train['type_sig']
    le = preprocessing.LabelEncoder()
    X = X.applymap(lambda x: x.real)
    
    y1 = le.fit_transform(y)
    X_train_in, X_test_in, y_train_in, y_test_in = train_test_split(
        X, y1, test_size=0.2, random_state=21,
        shuffle=True, stratify=y
    )

    model = digitsPerceptron(            
            input_dim=len(X.columns), 
            hidden_dim=300
        ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    X_train_in.reset_index(drop=True,inplace = True)
    X_test_in.reset_index(drop=True,inplace = True)     
    X_train = torch.tensor(X_train_in.values, dtype=torch.float).to(device)
    y_train = torch.tensor(y_train_in, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test_in.values, dtype=torch.float).to(device)
    y_test = torch.tensor(y_test_in, dtype=torch.long).to(device)

    EPOCHS  = 500
    loss_list = np.zeros((EPOCHS,))
    accuracy_list = np.zeros((EPOCHS,))
    for epoch in tqdm(range(EPOCHS)):
        model.train()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        loss_list[epoch] = loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test)
            correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.float)
            accuracy_list[epoch] = correct.mean().item() 
    model.eval()

    with torch.no_grad():
        y_pred = np.argmax(model(X_test).detach().cpu().numpy(), axis=1)
    #####################################
    
    
    neiro_acc[SNR] = accuracy_score(y_test_in, y_pred)

        
    


  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

CPU times: user 22min 27s, sys: 10.1 s, total: 22min 37s
Wall time: 22min 48s


In [ ]:
X_test[:1]

tensor([[ 2.7520e-01,  6.2601e-05,  1.0030e+00, -2.6908e-01,  2.7418e-01,
          2.7418e-01, -1.0017e+00,  6.2601e-05,  1.0030e+00, -9.1335e-01,
         -3.0180e+00, -1.0022e+00]], device='cuda:0')

In [ ]:
%%timeit -n100 -r3
with torch.no_grad():
        y_pred = np.argmax(model(X_test[:1]).detach().cpu().numpy(), axis=1)

253 µs ± 33.4 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [ ]:
cat = CatBoostClassifier(silent=True, iterations = 800 )

In [ ]:
cat.fit(X_train_in, y_train_in,plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
%%timeit -n100 -r3
cat.predict(X_test_in[:1])

1.06 ms ± 46.7 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [ ]:
pd.DataFrame(neiro_acc,index = ['neiro']).T.to_csv('new_neiro_300.csv')
from google.colab import files
files.download('new_neiro_300.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_p = pd.DataFrame(neiro_acc,index = ['neiro']).T
import plotly.express as px
fig = px.line(df_p)
fig.show()